In [1]:
import serial
from threading import Thread
import struct
import time
import numpy as np

In [3]:
class LoadCell:
    def __init__(self, port = '/dev/ttyACM0'):
        self.port = port
        self.serial = serial.Serial(port, baudrate=115200, timeout=1)
        self.running = True
        self.state = None
        self.offset = 0.
        self.thread = Thread(target=self.update)
        self.thread.start()

    def update(self):
        while self.running:
            data = self.serial.read_until(b'abc\n')
            # print(len(data))
            data = struct.unpack('1f',data[:-4])
            self.state = {'loadcell':data[0]-self.offset}
            
    def read(self):
        return self.state
    
    def close(self):
        self.running=False
        self.serial.close()
    
    def zeroReset(self):
        print('Calibrating, wait for 5 seconds ...')
        self.offset = 0.
        offset = [] 
        for i in range(500):
            time.sleep(0.01)
            offset.append(loadcell.read()['loadcell'])
            self.offset = np.array(offset).mean()

loadcell = LoadCell()



Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/rstaion/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_600004/1194936489.py", line 13, in update
  File "/home/rstaion/.local/lib/python3.8/site-packages/serial/serialutil.py", line 663, in read_until
    c = self.read(1)
  File "/home/rstaion/.local/lib/python3.8/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


In [4]:
loadcell.zeroReset()

Calibrating, wait for 5 seconds ...


In [28]:
loadcell.read()['loadcell']

14.136511153316379

In [35]:
import time
forces = []
for i in range(10000):
    forces.append(loadcell.read()['loadcell'])
    time.sleep(0.01)

KeyboardInterrupt: 

In [38]:
%matplotlib
import matplotlib.pyplot as plt
plt.plot(forces)
plt.grid(True)

# Calibration if Needed

In [ ]:
g = 9.81
mapping = np.array([
    [1*g, 30450.002791155868],
    [0.5*g, 14531.296401515152],
    [0.2*g, 6252.21525],
    [0.1*g, 2535.88125],
    [0.4*g, 11333.59525],
])

In [ ]:
p = np.polyfit(mapping[:,1], mapping[:,0],1)
p

In [ ]:
mapping[0,1]*p[0]+p[1]